# 1. torch.cuda 的核心功能

它主要负责 设备管理、张量与模型的迁移、同步控制，常见的功能包括：
| 方法 / 属性                          | 作用                                        |
| -------------------------------- | ----------------------------------------- |
| `torch.cuda.is_available()`      | 判断当前环境是否有可用的 GPU                          |
| `torch.cuda.device_count()`      | 返回可用 GPU 的数量                              |
| `torch.cuda.current_device()`    | 返回当前默认使用的 GPU id                          |
| `torch.cuda.get_device_name(id)` | 返回指定 GPU 的名称（如 "NVIDIA GeForce RTX 3080"） |
| `torch.cuda.set_device(id)`      | 设置当前进程使用的 GPU                             |
| `torch.cuda.empty_cache()`       | 释放未被使用的显存（PyTorch 会缓存显存以加快分配速度）           |
| `torch.cuda.memory_allocated()`  | 返回已分配的显存大小（字节）                            |
| `torch.cuda.memory_reserved()`   | 返回已保留的显存大小（字节）                            |

--- 
# 2. Tensor 与模型的 GPU 迁移

要让数据和模型在 GPU 上运行，需要`手动迁移`：

### （1）迁移 Tensor

In [ ]:
import torch

# 创建张量
x_cpu = torch.randn(3, 3)

# 迁移到 GPU（如果可用）
if torch.cuda.is_available():
    x_gpu = x_cpu.to("cuda")  
    # 等价写法：x_cpu.cuda()


### （2）迁移模型

In [ ]:
import torch.nn as nn

model = nn.Linear(10, 2)

# 将模型迁移到 GPU
if torch.cuda.is_available():
    model = model.to("cuda")

# 输入张量也要在同一个设备
x = torch.randn(5, 10).to("cuda")
output = model(x)


⚠️ 注意：模型和输入必须在同一个设备，否则会报错。

---
# 3. 指定 GPU 设备

如果你有多块 GPU，可以指定使用哪块：

In [ ]:
device = torch.device("cuda:1")  # 第二块 GPU（从 0 开始）
x = torch.randn(3, 3, device=device)

model = nn.Linear(10, 2).to(device)


也可以用上下文管理器：

In [ ]:
with torch.cuda.device(1):
    y = torch.randn(2, 2).cuda()  # 自动分配到 GPU:1


# 4. GPU 与 CPU 之间的切换

- 迁移到 CPU：

In [ ]:
x_cpu = x_gpu.to("cpu")
# 等价：x_gpu.cpu()


- ⚠️自动选择设备（`推荐写法`）：
- 这样代码能在 有 GPU 时用 GPU，没有 GPU 时用 CPU，提高通用性。

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

x = torch.randn(3, 3).to(device)
model = nn.Linear(10, 2).to(device)


# 5. 多 GPU 并行

在多 GPU 上训练时，可以用：

- `torch.nn.DataParallel(model)`（较旧，简单，但速度较慢）

- `torch.nn.parallel.DistributedDataParallel(model)`（`推荐`，分布式训练标准）

示例：

In [ ]:
if torch.cuda.device_count() > 1:
    print("使用多 GPU 训练:", torch.cuda.device_count(), "块 GPU")
    model = nn.DataParallel(model)

model.to("cuda")


# 6. 显存管理

训练时显存很容易爆掉，常用方法：

In [ ]:
torch.cuda.empty_cache()  # 释放显存
torch.cuda.memory_summary(device=None, abbreviated=False)  # 显存使用情况


## ✅ 总结：
torch.cuda 就是 PyTorch 与 GPU 交互的桥梁，主要功能有：

- 1、设备管理（选择 GPU，查询信息）

- 2、数据与模型迁移（CPU ↔ GPU）

- 3、显存管理（缓存、释放、监控）

- 4、多 GPU 并行训练